In [359]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns ##ploting and visual
import scipy.stats as stat
import numpy as np
import nltk.corpus
#nltk.download('stopwords')
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None 
import datetime

%matplotlib inline
from scipy import stats
from sklearn.metrics.pairwise import linear_kernel # for cosine similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [360]:
df= pd.read_csv("games.csv")
df.shape

(18800, 6)

In [361]:
#Getting rid of duplicates and DLCs (downloadable content and expansion to a current game)

In [362]:
df.drop_duplicates('name',keep='first', inplace=True)

df.reset_index(drop=True, inplace=True)

df = df[df["summary"].str.contains("spin-off|DLC|Expansion|pack|Pack") == False]
df.shape

(11322, 6)

In [363]:
#Changing the release date to year so I can have just the year for simplicity purposes 

In [364]:
df['year'] = pd.DatetimeIndex(df['release_date']).year

In [365]:
#changing year to a float for simplicty 

In [366]:
df["year"] = df["year"].astype(str).astype(float)

In [367]:
#The user can input what console they don't want 

In [446]:
val = input("please type what consoles, you DON'T want for example: Dreamcast | Playstation Vita and then hit ENTER")
print(val)

please type what consoles, you DON'T want for example: Dreamcast | Playstation Vita and then hit ENTER Dreamcast | Playstation 1


Dreamcast | Playstation 1


In [447]:
#Dropping consoles 

In [448]:
df = df[~df.platform.str.contains(val)]

In [449]:
#The user can now enter what starting year of games they want 

In [450]:
val2 = input("What release years do you want and hit ENTER")
val2 = int(val2)
print(val2)

What release years do you want and hit ENTER 2005


2005


In [373]:
#Dropping the games 

In [451]:
condition1 = df['year'] >= val2
true_index = condition1[condition1 == True].index
df =df[condition1]

In [452]:
#Getting of punctuation and renaming summary to s to keep it simple 

In [453]:
df.rename(columns={'summary': 's'}, inplace=True)
df['s']=df['s'].str.lower()
df['s'] = df['s'].str.replace(r'[^\w\s]+', '')

/var/folders/l8/z5jtbjgx7xg8pwl0g1mdd2fh0000gn/T/ipykernel_33311/16167914.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['s'] = df['s'].str.replace(r'[^\w\s]+', '')


In [454]:
#Getting rid of null rows 

In [455]:
null_rows = df[df['s'].isnull()].index

df.drop(index=null_rows, inplace=True)

In [456]:
#Gettingg rid of stopwords 

In [457]:
stop_words = stopwords.words('english')
df['s'] = df['s'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
#print(df['s'])

In [458]:
#Turning User Review and Meta Score into floats to be more compatible 

In [459]:
df = df.drop(df[df['user_review'] == 'tbd'].index)
df["user_review"] = df["user_review"].astype(str).astype(float)
df["meta_score"] = df["meta_score"].astype(str).astype(float)

In [460]:
#Getting games with 65 score and better

In [461]:
condition = df['meta_score'] > 65
true_index = condition[condition == True].index
df =df[condition]

In [462]:
df.reset_index(drop=True, inplace=True)

df.index

RangeIndex(start=0, stop=4387, step=1)

In [463]:
#First I did TF-IDF Vectorizer
tfidf = TfidfVectorizer()

#Created the matrix
tfidf_matrix = tfidf.fit_transform([str(i) for i in df['s']])

#Output the shape of matrix
tfidf_matrix.shape

(4387, 23486)

In [464]:
cosine = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine

array([[1.        , 0.00936336, 0.01339456, ..., 0.01698337, 0.        ,
        0.        ],
       [0.00936336, 1.        , 0.10136275, ..., 0.00153726, 0.00724332,
        0.        ],
       [0.01339456, 0.10136275, 1.        , ..., 0.0053351 , 0.01196047,
        0.01506316],
       ...,
       [0.01698337, 0.00153726, 0.0053351 , ..., 1.        , 0.00624388,
        0.03055552],
       [0.        , 0.00724332, 0.01196047, ..., 0.00624388, 1.        ,
        0.00358405],
       [0.        , 0.        , 0.01506316, ..., 0.03055552, 0.00358405,
        1.        ]])

In [465]:
# reverse the indices and video game titles and combine Names and Meta Score
df["meta_score"] = df["meta_score"].astype(str)
#EC = Empty space that way there can be a space between name and score 
df["EC"] = " "
df["Both Name and Score"] = df["name"] + df["EC"] + df["meta_score"]
indices = pd.Series(df.index, index=df['name'])
indices

name
Super Mario Galaxy 2                               0
Red Dead Redemption 2                              1
Grand Theft Auto V                                 2
Disco Elysium: The Final Cut                       3
The Legend of Zelda: Breath of the Wild            4
                                                ... 
Naruto to Boruto: Shinobi Striker               4382
Bounty Train                                    4383
Adventure Time: Finn and Jake Investigations    4384
A.R.E.S. Extinction Agenda EX                   4385
Ghost Pirates of Vooju Island                   4386
Length: 4387, dtype: int64

In [466]:
def recommender_system(title, cosine=cosine):
    
    # get index of video game that matches title
    
    index = indices[title]
    
    # get pairwise similarity scores of all video games with the given title
    
    sim_scores = list(enumerate(cosine[index]))
    
    # sort games based on similarity scores
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # get scores of 10 most similar video games
    
    sim_scores = sim_scores[1:11]
    
    # get game indices
    
    game_indices = [i[0] for i in sim_scores]
    
    # return top 10 most similar video games
    
    recs = df['Both Name and Score'].iloc[game_indices]
    
    return recs

In [467]:
recommender_system("Mario Kart 8")

536                      Mario Kart 7 85.0
1175                  FAST Racing NEO 81.0
1065                 ModNation Racers 82.0
3044                Team Sonic Racing 73.0
1629                     Wipeout 2048 79.0
3288           Test Drive Unlimited 2 72.0
2514    Mario Kart Live: Home Circuit 75.0
592                    Project CARS 2 84.0
3109             FAST - Racing League 73.0
1591                   Hotshot Racing 79.0
Name: Both Name and Score, dtype: object

In [468]:
recommender_system("Assassin's Creed Rogue")

3394              Assassin's Creed Rogue Remastered 71.0
3245          Assassin's Creed: The Ezio Collection 72.0
1370         Assassin's Creed: The Rebel Collection 80.0
3898             Assassin's Creed Chronicles: China 69.0
3688                             Banner of the Maid 70.0
3333    Assassin's Creed Syndicate: Jack the Ripper 72.0
504                        Assassin's Creed Origins 85.0
4353                 Assassin's Creed Liberation HD 66.0
106                   Assassin's Creed: Brotherhood 90.0
1357                  Assassin's Creed: Revelations 80.0
Name: Both Name and Score, dtype: object